# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [30]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [31]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [32]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [33]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [34]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [35]:
# Check the result:

A @ x - b

array([ 1.11022302e-16,  0.00000000e+00, -2.22044605e-16, -1.11022302e-16,
        1.11022302e-16,  0.00000000e+00, -2.42861287e-17,  0.00000000e+00,
       -2.77555756e-17,  1.11022302e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [36]:
def Jacobi( A, b, iter ):

  x = np.ones( n )
  for _ in range( iter ):
    x = BB @ x + c

  return x

x = Jacobi( A, b, 100 )

print( A @ x - b )

#reducing elements of the diagonal by 1%
A1 = A - np.diag( A ) / 100
x1 = Jacobi( A1, b, 100 )

print( A1 @ x1 - b )



[ 1.11022302e-16  0.00000000e+00 -2.22044605e-16 -1.11022302e-16
  1.11022302e-16  0.00000000e+00 -2.42861287e-17  0.00000000e+00
 -2.77555756e-17  1.11022302e-16]
[-0.04242902 -0.04242902 -0.04242902 -0.04242902 -0.04242902 -0.04242902
 -0.04242902 -0.04242902 -0.04242902 -0.04242902]


# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [48]:

def gauss_seidel(A, b, max_iterations=10000):
    
    x = np.zeros_like(b, dtype=np.double)
    
    #Iterate
    for k in range(max_iterations):
        
        x_old  = x.copy()
        
        #Loop over rows
        for i in range(A.shape[0]):
            x[i] = (b[i] - np.dot(A[i,:i], x[:i]) - np.dot(A[i,(i+1):], x_old[(i+1):])) / A[i ,i]
            
    return x

x = gauss_seidel( A, b, 100 )

print( A @ x - b )


for iter in range(0, 20, 1):
  x = gauss_seidel( A, b, iter )
  print( iter, np.linalg.norm( A @ x - b) )

[ 0.00000000e+00 -1.11022302e-16  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.73472348e-17  0.00000000e+00
  0.00000000e+00 -1.11022302e-16]
0 2.0642489862459183
1 0.2921553884672586
2 0.020168897159971992
3 0.0006626917588744586
4 5.890910715462774e-05
5 3.512188242341035e-06
6 1.6509251992006011e-07
7 1.5276279921693323e-08
8 5.525532375812141e-10
9 5.396386130602174e-11
10 2.716449659983224e-12
11 1.5779115716809045e-13
12 1.2517128008334558e-14
13 5.924633063968824e-16
14 1.9506192212679625e-16
15 1.5796464744355657e-16
16 1.5796464744355657e-16
17 1.5796464744355657e-16
18 1.5796464744355657e-16
19 1.5796464744355657e-16


# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [51]:
def min_residual( A, b, iter ):

  x = np.ones( n )

  for _ in range( n_iter ):

    #compute previous residual
    residual = A @ x - b

    tau = np.dot( residual, A @ residual ) / np.dot( A @ residual, A @ residual )

    print( tau )
    x -= tau * residual

  return x

x = min_residual( A, b, 40 )
print( A @ x - b )

0.049458091915919704
0.06632815899696641
0.05646528371934232
0.059136861448334216
0.056149365850671734
0.05962056630691936
0.055321259622594446
0.06019139542967882
0.054726519624859894
0.06091835471785148
0.054788794259621576
0.06154317282841529
0.055177210974496134
0.06162125644817706
0.05514529510209265
0.06170111966684046
0.054725649763148976
0.06212081019990162
0.054002133608905656
0.06437401757338992
0.05401091580604399
0.06011645923749508
0.0627115085769417
0.06413230252109958
0.06413230252109958
0.06413230252109958
0.06413230252109958
0.06413230252109958
0.06413230252109958
0.06413230252109958
0.06413230252109958
0.06413230252109958
0.06413230252109958
0.06413230252109958
0.06413230252109958
0.06413230252109958
0.06413230252109958
0.06413230252109958
0.06413230252109958
0.06413230252109958
0.06413230252109958
0.06413230252109958
0.06413230252109958
0.06413230252109958
0.06413230252109958
0.06413230252109958
0.06413230252109958
0.06413230252109958
0.06413230252109958
0.0641323025